In [ ]:
from datetime import datetime, timedelta
import time
import pandas as pd
import calendar

In [ ]:
draws = pd.read_csv('base_de_datos.csv', dtype={'Sorteos': "string"}, parse_dates=['Dates'])

In [ ]:
df = pd.DataFrame(columns=['Dates', 'Sorteos', 'Nro1', 'Nro2', 'Nro3', 'Nro4', 'Nro5', 'Star_1', 'Star_2'])

print("""Bienvenido a tu base de datos. Dime, ¿quieres actualizar los datos que tienes?
1. Sí
2. No""")

answer = int(input(""))

while answer != 1:
    print("Esperando instrucciones")
    answer = int(input())
    
for date in draws['Dates']:
    audit_date_3_days = timedelta(days = 3)
    audit_date_4_days = timedelta(days = 4)
